In [37]:
from jupyter_dash import JupyterDash
import IPython

In [38]:
import dash
from dash import html
import dash_jbrowse
print(dash_jbrowse.__version__)

1.2.0


In [39]:
def register_notebook_callbacks():
    try:
        import google.colab.output # noqa
        _in_colab_session = True
        register_colab()
    except: # noqa
        _in_colab_session = False
    try:
        shell = IPython.get_ipython().__class__.__name__ # noqa
        if shell == 'ZMQInteractiveShell': # noqa
            _in_jupyter_session = True
        else:
            _in_jupyter_session = False
    except: # noqa
        _in_jupyter_session = False
    print("colab: ", _in_colab_session)
    print("jupyter: ", _in_jupyter_session)

def register_colab():
    def callback(path, start=None, end=None):
            if start != None:
                start = int(start)
            if end != None:
                end = int(end)
            binary_file_data = read_file(path, start, end)
            return base64.b64encode(binary_file_data)
            #return binary_file_data
    google.colab.output.register_callback('ColabLocalFile', callback)

def read_file(path, start=None, end=None):
    with open(path, 'rb') as binary_file:
        if start != None:
            size = end - start
            binary_file.seek(start)
            binary_file_data = binary_file.read(size)
        else:
            binary_file_data = binary_file.read()
    return binary_file_data

In [40]:
def launch_jbrowse(assembly, tracks, defaultSession, location, aggregateTextSearchAdapters, my_internet_accounts):
    app = JupyterDash(__name__)
    register_notebook_callbacks()
    app.layout = html.Div(
        [
            dash_jbrowse.LinearGenomeView(
                id="input",
                assembly=assembly,
                tracks=tracks,
                defaultSession=defaultSession,
                location=location,
                aggregateTextSearchAdapters=aggregateTextSearchAdapters,
                internetAccounts=my_internet_accounts
            ),
        ],
    )
    
    app.run_server(mode="external",height=300, port=3020)

In [41]:
my_assembly = {
    "name": "GRCh38",
    "sequence": {
        "type": "ReferenceSequenceTrack",
        "trackId": "GRCh38-ReferenceSequenceTrack",
        "adapter": {
            "type": "BgzipFastaAdapter",
            "fastaLocation": {
                "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/fasta/GRCh38.fa.gz",
                "locationType": "UriLocation",
                "internetAccountId": "jupyterLocalFile"
            },
            "faiLocation": {
                "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/fasta/GRCh38.fa.gz.fai",
            },
            "gziLocation": {
                "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/fasta/GRCh38.fa.gz.gzi",
            },
        },
    },
    "aliases": ["hg38"],
    "refNameAliases": {
        "adapter": {
            "type": "RefNameAliasAdapter",
            "location": {
                "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/hg38_aliases.txt",
            },
        },
    },
}
my_internet_accounts = [
    {
      "type": "ColabLocalFileInternetAccount",
      "internetAccountId": "colabLocalFile",
      "name": "Colab",
      "description": "Account to access local files in Colab",
    },
    {
      "type": "JupyterNotebookLocalFileInternetAccount",
      "internetAccountId": "jupyterLocalFile",
      "name": "Jupyter",
      "description": "Account to access local files in Jupyter notebook",
    },
]
my_tracks = [
    {
        "type": "FeatureTrack",
        "trackId": "ncbi_refseq_109_hg38",
        "name": "NCBI RefSeq (GFF3Tabix)",
        "assemblyNames": ["GRCh38"],
        "category": ["Annotation"],
        "adapter": {
            "type": "Gff3TabixAdapter",
            "gffGzLocation": {
                "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/ncbi_refseq/GCA_000001405.15_GRCh38_full_analysis_set.refseq_annotation.sorted.gff.gz",
                "locationType": "UriLocation",
                "internetAccountId": "jupyterLocalFile"
            },
            "index": {
                "location": {
                    "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/ncbi_refseq/GCA_000001405.15_GRCh38_full_analysis_set.refseq_annotation.sorted.gff.gz.tbi",
                },
            },
        },
    },
]

my_default_session = {
    "name": "My session",
    "view": {
        "id": "linearGenomeView",
        "type": "LinearGenomeView",
        "tracks": [
            {
                "type": "FeatureTrack",
                "configuration": "ncbi_refseq_109_hg38",
                "displays": [
                    {
                        "type": "LinearBasicDisplay",
                        "configuration": "ncbi_refseq_109_hg38-LinearBasicDisplay",
                    },
                ],
            },
        ],
    },
}
my_aggregate_text_search_adapters=[
    {
        "type": "TrixTextSearchAdapter",
        "textSearchAdapterId": "hg38-index",
        "ixFilePath": {
            "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/trix/hg38.ix"
        },
        "ixxFilePath": {
            "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/trix/hg38.ixx"
        },
        "metaFilePath": {
            "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/GRCh38/trix/meta.json"
        },
        "assemblyNames": ['GRCh38'],
    }
]
my_location = "7:140,713,328-140,924,929"


In [42]:
launch_jbrowse(my_assembly, my_tracks, my_default_session, my_location, my_aggregate_text_search_adapters,my_internet_accounts)

colab:  False
jupyter:  True
Dash app running on http://127.0.0.1:3020/


In [9]:
def launch_jbrowse_cvg(assembly, tracks, defaultSession, aggregateTextSearchAdapters):
    app = JupyterDash(__name__)

    app.layout = html.Div(
        [
            dash_jbrowse.CircularGenomeView(
                id="input",
                assembly=assembly,
                tracks=tracks,
                defaultSession=defaultSession,
                aggregateTextSearchAdapters=aggregateTextSearchAdapters
            ),
        ],
    )
    
    app.run_server(mode="inline",height=600, port=3003)

In [10]:
my_assembly = {
    "name": "hg19",
    "aliases": ["GRCh37"],
    "sequence": {
        "type": "ReferenceSequenceTrack",
        "trackId": "Pd8Wh30ei9R",
        "adapter": {
            "type": "BgzipFastaAdapter",
            "fastaLocation": {
                "uri": "https://jbrowse.org/genomes/hg19/fasta/hg19.fa.gz",
                "locationType": 'UriLocation',
            },
            "faiLocation": {
                "uri": "https://jbrowse.org/genomes/hg19/fasta/hg19.fa.gz.fai",
                "locationType": 'UriLocation',
            },
            "gziLocation": {
                "uri": "https://jbrowse.org/genomes/hg19/fasta/hg19.fa.gz.gzi",
                "locationType": 'UriLocation',
            },
        },
    },
    "refNameAliases": {
        "adapter": {
            "type": "RefNameAliasAdapter",
            "location": {
                "uri": "https://s3.amazonaws.com/jbrowse.org/genomes/hg19/hg19_aliases.txt",
                "locationType": 'UriLocation',
            },
        },
    },
}

my_tracks = [
    {
        "type": "VariantTrack",
        "trackId": "pacbio_sv_vcf",
        "name": "HG002 Pacbio SV (VCF)",
        "assemblyNames": ["hg19"],
        "category": ["GIAB"],
        "adapter": {
            "type": "VcfTabixAdapter",
            "vcfGzLocation": {
                "uri": 'https://s3.amazonaws.com/jbrowse.org/genomes/hg19/pacbio/hs37d5.HG002-SequelII-CCS.bnd-only.sv.vcf.gz',
                "locationType": 'UriLocation',
            },
            "index": {
                "location": {
                    "uri": 'https://s3.amazonaws.com/jbrowse.org/genomes/hg19/pacbio/hs37d5.HG002-SequelII-CCS.bnd-only.sv.vcf.gz.tbi',
                    "locationType": 'UriLocation',
                },
            },
        },
    },
]

my_default_session = {
    "name": "My session",
    "view": {
        "id": "circularView",
        "type": "CircularView",
        "bpPerPx": 5000000, 
    },
}


my_theme = {
    "theme": {
        "palette": {
            "primary": {
                "main": "#311b92",
            },
            "secondary": {
                "main": "#0097a7",
            },
            "tertiary": {
                "main": "#f57c00",
            },
            "quaternary": {
                "main": "#d50000",
            },
            "bases": {
                "A": {"main": "#98FB98"},
                "C": {"main": "#87CEEB"},
                "G": {"main": "#DAA520"},
                "T": {"main": "#DC143C"},
            },
        },
    },
}
